In [2]:
import openai
import json
import numpy as np
from numpy.linalg import norm

api_key = open("api_key").read()
openai.api_key = api_key

In [3]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def cos_sim(a, b):
   return np.dot(a, b)/(norm(a)*norm(b))

def save_json(data, filepath=r'new_data.json'):
   with open(filepath, 'w') as fp:
      json.dump(data, fp, indent=4)

In [ ]:
def merge_paragraphs(sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [ ]:
event_dict = json.load(open("data/result/RAMS/gpt_biHgraph_dev/hyperedges.json"))

candidates = [
    '0-Compare-Require-Inable-Problem',
    '1-Occurred',
    '2-Involve-Investigate-Lose',
    '3-Bombing-Surrender-Celebrate',
    '4-Appointment-Question-Lack-Link',
    '5-Mention-Criticize',
    '6-Describe-Imprison-Convict-Punish',
]
# candidates = ['skirmish-Fans-Fans-French port city of Marseille-bottles , chairs and other objects', 'surrendered-Q17', 'bought-appointment-commodities trader', 'agreement-Obama administration-Q212', 'walked-Q868772-we', 'guilty-Q3702004-pay - for - play campaign finance scheme', 'convicted-6,267 inmates-Q1886845-drug - related crimes']
event_data_list = [event_dict[candidate] for candidate in candidates]

In [ ]:
candidate_paragraphs = { candidate: merge_paragraphs(event_dict[candidate]['content']) for candidate in candidates }

In [6]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))

In [7]:
# All The News
for article in All_articles:
    try:
        article['embedding'] = get_embedding(article['summary'])
    except:
        continue
save_json(All_articles, "data/raw/AllTheNews/embeddings/2016_10p.json")

In [4]:
All_articles = json.load(open("data/raw/AllTheNews/events/2016_10p.json"))
All_article_embeddings = json.load(open("data/raw/AllTheNews/embeddings/2016_10p.json"))
print(len(All_article_embeddings), len(All_articles))

7548 7548


In [ ]:
# RAMS
# embeddings_dict = {candidate: get_embedding(candidate_paragraphs[candidate]) for candidate in candidates}
for event_id, event in event_dict.items():
    print(event_id, len(event_dict))
    event['embedding'] = get_embedding(merge_paragraphs(event['content']))
save_json(event_dict, "data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings.json")

In [ ]:
def reorder_hyperedge_id(hyperedges_w_embeddings_dict):
    ordered_dict = {}
    for key, value in hyperedges_w_embeddings_dict.items():
        id_split = key.split('-')
        doc_id = id_split[0]
        triggers = id_split[1:]
        triggers.sort()
        ordered_dict[doc_id + '-' + '-'.join(triggers)] = value
    return ordered_dict
hyperedges_w_embeddings_dict = json.load(open("data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings.json"))
ordered_dict = reorder_hyperedge_id(hyperedges_w_embeddings_dict)
save_json(ordered_dict, 'data/result/RAMS/gpt_biHgraph_dev/hyperedges_w_embeddings_ordered.json')